In [1]:
import numpy as np
from scipy.stats import binom
from matplotlib import pyplot as plt
%matplotlib inline


In [2]:
# A Probability Distribution; an {outcome: probability} mapping.

class ProbDist(dict):
    def __init__(self, mapping=(), **kwargs):
        self.update(mapping, **kwargs)
        total = sum(self.values())
        for outcome in self:
            self[outcome] = self[outcome] / total
            assert self[outcome] >= 0

In [3]:
#The probability of an event, given a sample space of outcomes. 
#event: a collection of outcomes, or a predicate that is true of outcomes in the event. 
#space: a set of outcomes or a probability distribution of {outcome: frequency} pairs.

def p(event, space): 
    if is_predicate(event):
        event = such_that(event, space)
    if isinstance(space, ProbDist):
        return sum(space[o] for o in space if o in event)
    else:
        return Fraction(len(event & space), len(space))

is_predicate = callable

In [4]:
def such_that(predicate, space): 
    if isinstance(space, ProbDist):
        return ProbDist({o:space[o] for o in space if predicate(o)})
    else:
        return {o for o in space if predicate(o)}

### Scores Taken From Point Table Before Singapore Grand Prix

In [5]:
%autosave 10
SGP = ProbDist(
    LH = 284, VB=221,MV=185,CL=182,SV=169,PG=65,CS=58,DR=34,AA=34,DK=33,NH=31,
    KR=31,SP=27,LN=25,LS=19,KM=18,RG=8,AG=3,RK=1,GR=0)
SGP

Autosaving every 10 seconds


{'LH': 0.19887955182072828,
 'VB': 0.15476190476190477,
 'MV': 0.12955182072829133,
 'CL': 0.12745098039215685,
 'SV': 0.11834733893557423,
 'PG': 0.04551820728291316,
 'CS': 0.04061624649859944,
 'DR': 0.023809523809523808,
 'AA': 0.023809523809523808,
 'DK': 0.023109243697478993,
 'NH': 0.021708683473389355,
 'KR': 0.021708683473389355,
 'SP': 0.018907563025210083,
 'LN': 0.01750700280112045,
 'LS': 0.01330532212885154,
 'KM': 0.012605042016806723,
 'RG': 0.0056022408963585435,
 'AG': 0.0021008403361344537,
 'RK': 0.0007002801120448179,
 'GR': 0.0}

### Scores Taken From Point Table Before Russian Grand Prix

In [6]:
RGP = ProbDist(
    LH = 296, VB=231,MV=200,CL=200,SV=194,PG=69,CS=58,DR=34,AA=42,DK=33,NH=33,
    KR=31,SP=27,LN=31,LS=19,KM=18,RG=8,AG=4,RK=1,GR=0)
    


### Joint probablity for both the races

In [7]:

c = 0;
def joint(A, B, sep=''):
    return ProbDist({a + sep + b: A[a] * B[b]
                    for a in A
                    for b in B})

MM = joint(SGP, RGP ,'')


### Probability Distribution for each F1 driver to win both the Singaporean and Russian Grand Prix

In [8]:
# Filtering the winners in both the races 
# Comparing the first two places in the "dictionary Key" with the last two places
# so in first element key is LHLH
# Therefore  position 0 is L. Position 1 is H and So on.
# Thus if position 0 and position 1 equals to position 2 and position 3 then it should print the value
for a in MM:
    if(a[0]+a[1] == a[2]+a[3]):
        print({a : MM[a]})


{'LHLH': 0.038501208200742654}
{'VBVB': 0.023381294964028757}
{'MVMV': 0.016945954313707157}
{'CLCL': 0.01667115505456596}
{'SVSV': 0.01501594751700548}
{'PGPG': 0.0020541244620804486}
{'CSCS': 0.0015407078462516452}
{'DRDR': 0.0005294465726120397}
{'AAAA': 0.0006540222367560491}
{'DKDK': 0.0004987606553412728}
{'NHNH': 0.00046853273683574104}
{'KRKR': 0.00044013681339115075}
{'SPSP': 0.00033388109985655446}
{'LNLN': 0.0003549490430573796}
{'LSLS': 0.0001653375542499536}
{'KMKM': 0.00014839159993624643}
{'RGRG': 2.9311920975061026e-05}
{'AGAG': 5.495985182823942e-06}
{'RKRK': 4.5799876523532853e-07}
{'GRGR': 0.0}


### Probability for Mercedes to win both races

In [9]:
ans = MM["LHLH"] + MM["VBVB"] + MM["VBLH"] + MM["LHVB"]
print(ans)


0.12188950138590415


### Probability for Mercedes to win at least one race

In [10]:
c = 0;
for a in MM.keys():
    if a[0] + a[1]=="LH" or a[0] + a[1] == "VB" or a[2] + a[3] =="LH"  or a[2] + a[3] == "VB":
        #print({a : MM[a]})
        c = c + MM[a]
print(c )

0.5764216739671665


### If Mercedes wins the first race, probability that Mercedes wins the next one

In [11]:


def merc_wins_first_race(outcome): return outcome.startswith("LH") or outcome.startswith("VB")
such_that(merc_wins_first_race,MM)

def merc_wins_second(outcome): return  outcome.endswith('LH') or outcome.endswith('VB')
p(merc_wins_second, such_that(merc_wins_first_race, MM))


0.3446697187704381

### Mercedes wins at least one of these two races, probability Mercedes wins both races

In [12]:

#First we create a space where it contains every possibility mercedes winning the first race or second race. 
#From that space we select the possibility where mercedes wins the both the races
#Similarly we do it for ferrari redbull & Renault
def merc_wins_atleast_one_of_two(outcome): return outcome.startswith("LH") or outcome.startswith("VB") or outcome.endswith("LH") or outcome.endswith("VB")
a = such_that(merc_wins_atleast_one_of_two,MM) 

def merc_wins_both_race(outcome): return outcome.startswith("LHLH") or outcome.startswith("VBVB") or outcome.startswith("LHVB") or outcome.startswith("VBLH")
p(merc_wins_both_race, such_that(merc_wins_atleast_one_of_two,MM))


0.21145891435173045

### Ferrari wins at least one of these two races, probability Mercedes wins both races

In [13]:
#Ferrari
def ferrari_wins_atleast_one_of_two(outcome): return outcome.startswith("CL") or outcome.startswith("SV") or outcome.endswith("CL") or outcome.endswith("SV")
such_that(ferrari_wins_atleast_one_of_two,MM) 

def ferrari_wins_both_race(outcome): return outcome.startswith("CLCL") or outcome.startswith("SVSV") or outcome.startswith("SVCL") or outcome.startswith("CLSV")
p(ferrari_wins_both_race, such_that(ferrari_wins_atleast_one_of_two,MM))

0.14390380190985178

### Redbull wins at least one of these two races, probability Mercedes wins both races

In [14]:
#Redbull
def redbull_wins_atleast_one_of_two(outcome): return outcome.startswith("MV") or outcome.startswith("AA") or outcome.endswith("MV") or outcome.endswith("AA")
such_that(redbull_wins_atleast_one_of_two,MM) 

def redbull_wins_both_race(outcome): return outcome.startswith("MVMV") or outcome.startswith("AAAA") or outcome.startswith("AAMV") or outcome.startswith("MVAA")
p(redbull_wins_both_race, such_that(redbull_wins_atleast_one_of_two,MM))


0.08446852153789515

### Renault wins at least one of these two races, probability Mercedes wins both races

In [15]:
#Renault
def renault_wins_atleast_one_of_two(outcome): return outcome.startswith("DR") or outcome.startswith("NH") or outcome.endswith("DR") or outcome.endswith("NH")
such_that(renault_wins_atleast_one_of_two,MM) 

def renault_wins_both_race(outcome): return outcome.startswith("DRDR") or outcome.startswith("NHNH") or outcome.startswith("NHDR") or outcome.startswith("DRNH")
p(renault_wins_both_race, such_that(renault_wins_atleast_one_of_two,MM))

0.02283619812695982

### Probability Mercedes wins both races in any weather conditions (Assumming 5)

In [16]:
#Assuming the events are equiprobable. Therefore the probablity for rain =  0.2 , sunny = 0.2, cloudy = 0.2, snowy = 0.2 , foggy = 0.2

rain = 0.2 #rainy
sunn = 0.2 #sunny
clou = 0.2 #cloudy
snow = 0.2 #snowy
fogg = 0.2 #foggy

weather = ProbDist(rain =0.2, sunn = 0.2 , clou = 0.2 , snow = 0.2 , fogg = 0.2)


In [17]:
#We create the joint probability of SGP and weather condition and Russia and Weather condition.
def joint(A, B, sep=''):
    return ProbDist({a + sep + b: A[a] * B[b]
                    for a in A
                    for b in B})

weather_sgp = joint(SGP, weather ,'')
weather_rgp= joint(RGP, weather ,'')

#Joining the above 2 probability
weather_sgp_rgp = joint(weather_sgp, weather_rgp ,'')

len(weather_sgp_rgp)

10000

In [18]:
# First we create a space which has the probablity of mercedes winning in rains

def merc_wins_one_of_two_rainy(outcome): return  outcome.startswith("LHrain") or outcome.startswith("VBrain") or outcome.endswith("LHrain") or outcome.endswith("VBrain") 
a = such_that(merc_wins_one_of_two_rainy,weather_sgp_rgp) 

a11 = a["LHrainLHrain"] + a["VBrainLHrain"]  + a["LHrainVBrain"] + a["VBrainVBrain"]

#from the space we select only those outcomes in which mercedes wins in any climate condition
def merc_wins_both_race(outcome): return (outcome.startswith("LH") or outcome.startswith("VB")) and (outcome[6] + outcome[7] == "VB" or outcome[6] + outcome[7] == "LH") 
ans = p(merc_wins_both_race, such_that(merc_wins_one_of_two_rainy,weather_sgp_rgp))
#subtract the doublly occured probability
ans1 = ans - a11
ans1

0.2893805802888808